## Install deps

In [ ]:
%pip install pandas
%pip install scikit-learn
%pip install catboost
%pip install seaborn

## Imports

In [ ]:
from numpy import absolute
from pandas import read_csv
from sklearn.model_selection import train_test_split
from sklearn.linear_model import Lasso

## Load dataset

In [ ]:
dataframe = read_csv("clear_all_data.csv", index_col=0)
dataframe.head()

## Define args and y

In [ ]:
X = dataframe.drop("price_per_m2", axis=1)
y = dataframe.price_per_m2


In [ ]:
X.head()

In [ ]:
y.head()

## Split to train and test dataframes

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.9)

## Define and train model

### Lasso regression

* ### Create CV for lasso regression model

In [ ]:
from sklearn.model_selection import RepeatedKFold
cv = RepeatedKFold(n_splits=10, n_repeats=3)

* ### Learn model

In [ ]:
from sklearn.linear_model import LassoCV
model = LassoCV(cv=cv, n_jobs=-1)
model.fit(X_train, y_train)

### Linear regression

In [ ]:
from sklearn.linear_model import LinearRegression
model = LinearRegression()
model.fit(X_train, y_train)

## Make prediction

In [ ]:
y_pred = model.predict(X_test)

## Plot of data

In [ ]:
from matplotlib import pyplot as plt
import pandas as pd
import seaborn as sns

plt.figure(figsize=(200, 10))
sns.lineplot(x=range(len(y_pred)), y=y_pred, color="blue", label="Prediction")
sns.lineplot(x=range(len(y_test)), y=y_test, color='red', label='Actual')
plt.xlabel('Index')
plt.ylabel('Predicted Value')
plt.title('Line Plot of Predicted Values')
plt.show()

## Check score of model

In [ ]:
model.score(X_test, y_test)

## MAE

In [ ]:
import numpy as np
from sklearn.metrics import mean_absolute_error


mse = mean_absolute_error(y_true=y_test, y_pred=y_pred)
rmse = np.sqrt(mse)
print("Root Mean Absolute Error:", rmse)


## MSE

In [ ]:
import numpy as np
from sklearn.metrics import mean_squared_error


mse = mean_squared_error(y_true=y_test, y_pred=y_pred)
rmse = np.sqrt(mse)
print("Root Mean Squared Error:", rmse)
